In [1]:
import numpy as np
import pandas as pd
import datetime
from tqdm import tqdm 

import warnings 
warnings.filterwarnings('ignore')

In [2]:
def factor(my_data):
    my_data["Y_open"] = my_data["open"].shift(1).fillna(0)
    my_data["Y_high"] = my_data["high"].shift(1).fillna(0)
    my_data["Y_low"] = my_data["low"].shift(1).fillna(0)
    my_data["Y_close"] = my_data["close"].shift(1).fillna(0)
    my_data["Y_close_2"] = my_data["close"].shift(2).fillna(0)
    my_data["True_high"] = np.maximum(my_data["high"],my_data["Y_high"])
    my_data["True_low"] = np.minimum(my_data["low"],my_data["Y_low"])
    my_data["True_inte"] = my_data["True_high"] -my_data["True_low"] 
    my_data["Aver_inte"]=None
    my_data["Aver_inte"][9:]= pd.Series(np.convolve(my_data["True_inte"], np.ones(10) /10, 'valid'))
    my_data["T_open"] = my_data["open"]
    my_data = my_data.drop(['open','close','high','low','volume','money'],axis = 1)
    my_data = my_data[10:]
    return my_data

In [3]:
def signal(data):
    day_first_idexs = data.drop_duplicates('date').index.values #取出相同date的第一个日期index 如0 240 480
    day_idexs = np.hstack([day_first_idexs,data.shape[0]]) #加上最后一个index 如0 240 480。。。57840（方便后面取每日区间）
    data_dfs=[]
    
    for day_begin, day_end in zip(day_idexs[:-1], day_idexs[1:]-7):#取每一天的区间
        tmp_df = data.iloc[day_begin: day_end, :]
#         print(tmp_df)
#         zip(tmp_df["low"],tmp_df["low_B"],tmp_df["high"],tmp_df["high_B"],tmp_df["close"],tmp_df["open"])
        tmp_df["stop_price"]=None
        tmp_df["signal"]=None
        tmp_df["stop_or_not"]=False
        direction = None
        stop_price=None
        for i in range(day_begin+1,day_begin+len(tmp_df["time"])):
            if direction == None:
                if tmp_df["low"][i]<=tmp_df["low_B"][i] and tmp_df["low"][i-1]>tmp_df["low_B"][i-1]:
                    direction = -1
                    tmp_df["signal"][i] = -1
                elif tmp_df["high"][i]>=tmp_df["high_B"][i] and tmp_df["high"][i-1]<tmp_df["high_B"][i-1]:
                    direction = 1
                    tmp_df["signal"][i] = 1
                    
            elif direction == 1:
                if stop_price == None:
                    stop_price = tmp_df["open"][i]*(1.000-0.005)
                    tmp_df["stop_price"][i]=stop_price
                else:
                    if tmp_df["low"][i]<=stop_price:
                        direction = None
                        tmp_df["signal"][i] = 0 
                        tmp_df["stop_or_not"][i]=True
                        tmp_df["stop_price"][i]=stop_price
                        stop_price = None
                        
                
            elif direction == -1:
                if stop_price ==None:
                    stop_price=tmp_df["open"][i]*(1.000+0.005)
                    tmp_df["stop_price"][i]=stop_price
                else:
                    if tmp_df["high"][i]>=stop_price:
                        direction = None
                        tmp_df["signal"][i] = 0  
                        tmp_df["stop_or_not"][i]=True
                        tmp_df["stop_price"][i]=stop_price
                        stop_price = None
                        
            
            if i ==day_begin+len(tmp_df["time"])-1:##收盘平仓
                tmp_df["signal"][i] = 0 
        
#             tmp_df["stop_price"]=tmp_df["stop_price"].fillna(method='ffill').fillna(0)
#         if day_begin ==270:
#             break
        data_dfs.append(tmp_df)
    """ 拼接得到新DataFrane """
    tmp_data = pd.concat(data_dfs)
    tmp_data = tmp_data.reset_index(drop=True)
    return tmp_data
    

In [4]:
#持仓计算
def position(data, shift_period=1):
    signal = data['signal']
    position = signal.fillna(method='ffill').shift(shift_period).fillna(0)
    data['position'] = position
    return data

In [5]:
def statistic_performance(data, r0=0.03, data_period=1440, 
                          is_consider_open=False,
                          is_consider_stop=False,
                          is_daily_close = False,
                          point = None,
                          comm=None, leverage=None):
    position = data['position']
    close = data['close']
    open_p = data['open']
    
    d_first = data['date'].values[0]
    d_last = data['date'].values[-1]
    d_period = datetime.datetime.strptime(d_last, '%Y-%m-%d') - datetime.datetime.strptime(d_first, '%Y-%m-%d')
    y_period = d_period.days / 365

    # 1. 初始化 
    hold_in_price =  close.shift(1).copy()
    hold_in_price[close.index[0]] = open_p.values[0]
    hold_out_price = close.copy()
              
    # 2. 若考虑持仓信号开盘才能操作
    if is_consider_open:
        # 仓位变动第一期的period_in_price为当日open
        hold_in_price[abs(position - position.shift(1).fillna(0))>0] = open_p
        # 仓位变动前最后一期的period_out_price为下一段的open (除了整个序列最后一天不变)
        hold_out_price[abs(position - position.shift(-1).fillna(0))>0] = open_p.shift(-1)
        hold_out_price[hold_out_price.index[-1]] = close.values[-1]
    
    #3.考虑止损+跳空 比较止损价和开盘价
    if is_consider_stop:
        for i in range(data.index[0],data.index[0]+len(data['close'])):
            if data["position"][i]<0 and data['stop_or_not'][i] == True:#只有止损平仓stop_or_not是True 15:00:00平仓是False
                hold_out_price[i] = max(data['stop_price'][i],data['open'][i])            
            elif data["position"][i]>0 and data['stop_or_not'][i] == True:
                hold_out_price[i] = min(data['stop_price'][i],data['open'][i])

    #4.是否收盘前平仓
    if is_daily_close:
        daily_last_idxes = data.drop_duplicates(subset='date',keep='last').index
        #距离每天最后一个时间点的距离 如15:00:00平仓 15:15:00截止  5m一段 hold_out_price.loc[daily_last_idxes.values-3]=close
        hold_out_price.loc[daily_last_idxes.values] = close 
    
    #5.考虑指数点
    if point:
        # 仓位变动第一期的period_in_price long加指数点 short减指数点
        hold_in_price[position < position.shift(1).fillna(0)] = hold_in_price - point
        hold_in_price[position > position.shift(1).fillna(0)] = hold_in_price + point       
        # 仓位变动前最后一期的period_out_price long减指数点 short加指数点
        hold_out_price[position < position.shift(-1).fillna(0)] = hold_out_price + point
        hold_out_price[position > position.shift(-1).fillna(0)] = hold_out_price - point
           
    # 6. 计算收益
    hold_r = position * (hold_out_price/hold_in_price-1)

    # 7. 考虑换仓成本 
    if comm:
        chgpos_comm_perc = (1-comm)**abs(position - position.shift(1).fillna(0)) 
        hold_r = chgpos_comm_perc * (1+hold_r) - 1 
    # 8. 考虑杠杆 
    if leverage:
        hold_r *= leverage

    hold_win = hold_r>0
    hold_cumu_r = (1+hold_r).cumprod() - 1
    drawdown = (hold_cumu_r.cummax()-hold_cumu_r)/(1+hold_cumu_r).cummax()      
    ex_hold_r= hold_r-r0/(250*1440/data_period)
    data['hold_in_price'] = hold_in_price *abs(position)
    data['hold_out_price'] = hold_out_price *abs(position)
    data['hold_r'] = hold_r
    data['hold_win'] = hold_win
    data['hold_cumu_r'] = hold_cumu_r
    data['drawdown'] = drawdown
    data['ex_hold_r'] = ex_hold_r
    
    """       数值型特征 
        v_hold_cumu_r：         累计持仓收益
        v_pos_hold_times：      多仓开仓次数
        v_pos_hold_win_times：  多仓开仓盈利次数
        v_pos_hold_period：     多仓持有周期数
        v_pos_hold_win_period： 多仓持有盈利周期数
        v_neg_hold_times：      空仓开仓次数
        v_neg_hold_win_times：  空仓开仓盈利次数
        v_neg_hold_period：     空仓持有盈利周期数
        v_neg_hold_win_period： 空仓开仓次数
        
        v_hold_period：         持仓周期数（最后一笔未平仓订单也算）
        v_hold_win_period：     持仓盈利周期数（最后一笔未平仓订单也算）
        
        v_max_dd：              最大回撤
        v_annual_ret：          年化收益
        v_sharpe：              年化夏普
    """
    v_hold_cumu_r = hold_cumu_r.tolist()[-1]

    v_pos_hold_times= 0 
    v_pos_hold_win_times = 0
    v_pos_hold_period = 0
    v_pos_hold_win_period = 0
    v_neg_hold_times= 0 
    v_neg_hold_win_times = 0
    v_neg_hold_period = 0
    v_neg_hold_win_period = 0
    for w, r, pre_pos, pos in zip(hold_win, hold_r, position.shift(1), position):
        """ 当日有换仓（先结算上一次持仓，再初始化本次持仓） """
        if pre_pos!=pos: 
            # 判断pre_pos非空：若为空则是循环的第一次，此时无需结算，直接初始化持仓即可
            if pre_pos == pre_pos:
                # 结算上一次持仓
                if pre_pos>0:
                    v_pos_hold_times += 1
                    v_pos_hold_period += tmp_hold_period
                    v_pos_hold_win_period += tmp_hold_win_period
                    if tmp_hold_r>0:
                        v_pos_hold_win_times+=1
                elif pre_pos<0:
                    v_neg_hold_times += 1      
                    v_neg_hold_period += tmp_hold_period
                    v_neg_hold_win_period += tmp_hold_win_period
                    if tmp_hold_r>0:                    
                        v_neg_hold_win_times+=1
            # 初始化本次持仓
            tmp_hold_r = r
            tmp_hold_period = 0
            tmp_hold_win_period = 0
        else: 
            if abs(pos)>0:
                tmp_hold_period += 1
                if r>0:
                    tmp_hold_win_period += 1
                if abs(r)>0:
                    tmp_hold_r = (1+tmp_hold_r)*(1+r)-1       

    v_hold_period = (abs(position)>0).sum()
    v_hold_win_period = (hold_r>0).sum()
    v_max_dd = drawdown.max()    
    v_annual_std = ex_hold_r.std() * np.sqrt( len(data)/y_period ) 
    v_annual_ret = (1+v_hold_cumu_r) ** (1/y_period) - 1
    v_sharpe= v_annual_ret / v_annual_std

    """ 生成Performance DataFrame """
    performance_cols = ['累计收益', 
                        '多仓次数', '多仓成功次数', '多仓胜率', '多仓平均持有期', 
                        '空仓次数', '空仓成功次数',  '空仓胜率', '空仓平均持有期', 
                        '周期胜率', '最大回撤', '年化收益/最大回撤',
                        '年化收益','年化标准差', '年化夏普'
                       ]
    performance_values = ['{:.2%}'.format(v_hold_cumu_r),
                          v_pos_hold_times, v_pos_hold_win_times,
                                            '{:.2%}'.format(v_pos_hold_win_times/v_pos_hold_times if v_pos_hold_times != 0 else 0), 
                                            '{:.2f}'.format(v_pos_hold_period/v_pos_hold_times if v_pos_hold_times != 0 else 0),
                          v_neg_hold_times, v_neg_hold_win_times,
                                            '{:.2%}'.format(v_neg_hold_win_times / v_neg_hold_times if v_neg_hold_times != 0 else 0), 
                                            '{:.2f}'.format(v_neg_hold_period / v_neg_hold_times if v_neg_hold_times != 0 else 0),
                          '{:.2%}'.format(v_hold_win_period/v_hold_period if v_neg_hold_period != 0 else 0), 
                          '{:.2%}'.format(v_max_dd), 
                          '{:.2f}'.format(v_annual_ret/v_max_dd),
                          '{:.2%}'.format(v_annual_ret), 
                          '{:.2%}'.format(v_annual_std),
                          '{:.2f}'.format(v_sharpe)
                         ]
    performance_df = pd.DataFrame(performance_values, index=performance_cols)
    
    return data, performance_df, v_hold_cumu_r

## 测试

In [16]:
#读取数据
my_data_1d=pd.read_csv(
    'IF8888.CCFX_1d.csv', sep=',', parse_dates=True)
my_data_2m=pd.read_csv(
    'IF8888.CCFX_2m.csv', sep=',', parse_dates=True)
my_data_1d = my_data_1d[my_data_1d["date"]<='2013-02-01']
my_data_2m = my_data_2m[my_data_2m["date"]<='2013-02-01']
my_data_2m = my_data_2m.iloc[:, 0:7]
my_data = my_data_1d[my_data_1d["date"]<='2011-12-31']
my_data_1 = my_data_2m[my_data_2m["date"]<='2011-12-31']
# my_data_1d

In [7]:
def f(data,ind):
    y= []
    x =data[['Y_open', 'Y_high', 'Y_low', 'Y_close', 'Y_close_2','True_high','True_low','True_inte','Aver_inte']].values
    for x1,x2,x3,x4,x5,x6,x7,x8,x9 in zip(x[:, 0],x[:,1], x[:, 2],x[:, 3],x[:, 4],x[:, 5],x[:, 6],x[:, 7],x[:, 8]):
        tmp = ind[0]*x1+ind[1]*x2+ind[2]*x3+ind[3]*x4+ind[4]*x5+ind[5]*x6+ind[6]*x7+ind[7]*x8+ind[8]*x9
        if tmp>=0:#防止区间为负数
            y.append(tmp)
        else:
            y.append(10000) 
    return y

In [8]:
def bound_1(data,ind):
    tmp_data=pd.DataFrame()
#     print(f(data))
    tmp_data["date"]=data["date"]
#     tmp_data["open_B"]=data['T_open']
    tmp_data["B"]=f(data,ind)
    tmp_data["low_B"]=data["T_open"]-f(data,ind)
    tmp_data["high_B"]=data["T_open"]+f(data,ind)
    return tmp_data

In [9]:
ind =[-0.34,0.65,0.31,-1.25,0.81,-0.99,0.79,0.52,1.21]
my_data=factor(my_data)
my_data_2 =bound_1(my_data,ind)

# my_data_2
data_all = pd.merge(my_data_1,my_data_2,on='date')
my_signal = signal(data_all)
my_position = position(my_signal)
res_my_data, performance_df ,hold_cumu_r= statistic_performance(my_position, 
                                                   data_period=1, 
                                                   is_consider_open=True,
                                                   is_consider_stop = True,                       
                                                   point=0.4,
                                                   is_daily_close=True,
                                                   comm=0.00003)
print(performance_df)
# res_my_data.to_csv('10_1.csv', index=False)

                0
累计收益       28.60%
多仓次数           95
多仓成功次数         35
多仓胜率       36.84%
多仓平均持有期     45.51
空仓次数          124
空仓成功次数         54
空仓胜率       43.55%
空仓平均持有期     48.87
周期胜率       50.08%
最大回撤       10.30%
年化收益/最大回撤    1.58
年化收益       16.27%
年化标准差       8.93%
年化夏普         1.82


In [152]:
# res_my_data.query('B==10000')

In [67]:
# my_position[51040:51090]

In [68]:
# my_signal.query("signal ==1 or signal ==-1")

In [69]:
# my_signal[my_signal["stop_or_not"]]

In [70]:
# my_signal.query("stop_price!=0")

In [71]:
# import random
# random.uniform(-2,2)
# target = list('tobeornottobe')
# target = [ord(item) for item in target]
# print(target)

In [12]:
my_data=factor(my_data)
# 遗传算法
import numpy as np
import matplotlib.pyplot as plt
from deap import base, tools, creator, algorithms
import random
# 定义
creator.create('FitnessMax', base.Fitness, weights=(1.0,)) #　最大化问题
creator.create('Individual', list, fitness=creator.FitnessMax)

paraLength = 9
toolbox = base.Toolbox()
toolbox.register('param',random.uniform, -2, 2)
toolbox.register('individual', tools.initRepeat, creator.Individual, toolbox.param, n=paraLength)

# def f(data,ind):
#     y= []
#     x =data[['Y_open', 'Y_high', 'Y_low', 'Y_close', 'Y_close_2','True_high','True_low','True_inte','Aver_inte']].values
#     for x1,x2,x3,x4,x5,x6,x7,x8,x9 in zip(x[:, 0],x[:,1], x[:, 2],x[:, 3],x[:, 4],x[:, 5],x[:, 6],x[:, 7],x[:, 8]):
#         tmp = ind[0]*x1+ind[1]*x2+ind[2]*x3+ind[3]*x4+ind[4]*x5+ind[5]*x6+ind[6]*x7+ind[7]*x8+ind[8]*x9
#         if tmp>=0:#防止区间为负数
#             y.append(tmp)
#         else:
#             y.append(10000)  
#     return y
# def bound(data,ind):
#     tmp_data=pd.DataFrame()
# #     print(f(data))
#     tmp_data["date"]=data["date"]
#     tmp_data["open_B"]=data['T_open']
#     tmp_data["B"]=f(data,ind)
#     tmp_data["low_B"]=data["T_open"]-f(data,ind)
#     tmp_data["high_B"]=data["T_open"]+f(data,ind)
#     return tmp_data
    
# 评价函数 -- 累计收益率
def evaluate(ind): 
#     print(ind)
    my_data_2 =bound_1(my_data,ind)
#     print(my_data_2)
    data_all = pd.merge(my_data_1,my_data_2,on='date')
    my_signal = signal(data_all)
    my_position = position(my_signal)
    res_my_data, performance_df, hold_cumu_r = statistic_performance(my_position, 
                                                   data_period=1, 
                                                   is_consider_open=True,
                                                   is_consider_stop = True,                       
                                                   point=0.4,
                                                   is_daily_close=True,
                                                   comm=0.00003)
#     print(hold_cumu_r)
    return (hold_cumu_r),

toolbox.register('evaluate', evaluate)


# 记录迭代数据
stats = tools.Statistics(key=lambda ind:ind.fitness.values)
stats.register('avg', np.mean)
stats.register('max', np.max)
stats.register('min', np.min)
stats.register('std', np.std)
logbook = tools.Logbook()
logbook.header = ['gen', 'nevals'] + (stats.fields)

# 生成初始族群
toolbox.register('population', tools.initRepeat, list, toolbox.individual)
popSize = 500
pop = toolbox.population(popSize)
# 注册遗传算法操作 -- 选择,交叉,突变
toolbox.register('select', tools.selTournament, tournsize=2)
toolbox.register('mate', tools.cxUniform, indpb=0.5)
toolbox.register('mutate', tools.mutShuffleIndexes, indpb=0.5)

# 遗传算法
ngen = 30 # 迭代步数
# CXPB  is the probability with which two individuals are crossed
# MUTPB is the probability for mutating an individual
cxpb = 0.8 # 交叉概率
mutpb = 0.2 # 突变概率

# 评价初始族群
invalid_ind = [ind for ind in pop if not ind.fitness.valid]
fitnesses = toolbox.map(toolbox.evaluate, pop)
for fitness, ind in zip(fitnesses, invalid_ind):
    ind.fitness.values = fitness

# 记录数据
record = stats.compile(pop)
logbook.record(gen=0, nevals=len(invalid_ind), **record)

# 迭代
for gen in range(1, ngen+1):
    # 配种选择
    offspring = toolbox.select(pop, popSize)
    offspring = [toolbox.clone(_) for _ in offspring]
    # 交叉
    for ind1, ind2 in zip(offspring[::2], offspring[1::2]):
        if random.random()<cxpb:
            toolbox.mate(ind1, ind2)
            del ind1.fitness.values
            del ind2.fitness.values
    # 突变
    for ind in offspring:
        if random.random()<mutpb:
            toolbox.mutate(ind)
            del ind.fitness.values
    
    # 评价子代 对于被改变的个体，重新计算其适应度
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for fitness, ind in zip(fitnesses, invalid_ind):
        ind.fitness.values = fitness
    # 环境选择 保留精英,保持种群规模
    combinedPop = pop + offspring
    pop = tools.selBest(combinedPop, popSize) # 精英保存策略
    # 记录数据
    # compile(sequence)# 将每个注册功能应用于输入序列数据，并将结果作为字典返回
    record = stats.compile(pop)
    logbook.record(gen=gen, nevals=len(invalid_ind), **record)
    print(logbook.stream)
    bestInd = tools.selBest(pop,1)[0]
    bestFit = bestInd.fitness.values[0]
    print('最优解为: '+str(bestInd))
    print('收益最大值为: '+str(bestFit))

# 可视化迭代过程
maxFit = logbook.select('max')
avgFit = logbook.select('avg')
plt.plot(maxFit, 'b-', label='Maximum Fitness')
plt.plot(avgFit, 'r-', label='Average Fitness')
plt.xlabel('# Gen')
plt.ylabel('Fitness')
plt.legend(loc='best')



KeyboardInterrupt: 

In [163]:
# 输出结果
bestInd = tools.selBest(pop,1)[0]
bestFit = bestInd.fitness.values[0]
print('最优解为: '+str(bestInd))
print('收益最大值为: '+str(bestFit))


最优解为: [0.6064531271027143, 1.1469476180530012, 1.2453036834458149, -1.657479142147305, -1.2606158379235208, 0.3857420334185466, -0.5111631710731213, 1.4795654027014211, -0.9825084296122464]
收益最大值为: 0.5956249931745368


In [17]:
# 最优解为: [1.1294917337831762, -1.9563977112441693, 0.8161140766278105, -0.6199366712270722, -0.02980181199055343, 1.439976758894478, -0.8013962459001673, 0.6302530667531654, -1.0354950655613822]
# 收益最大值为: 0.5891535611708241
# 最优解为: [0.6064531271027143, 1.1469476180530012, 1.2453036834458149, -1.657479142147305, -1.2606158379235208, 0.3857420334185466, -0.5111631710731213, 1.4795654027014211, -0.9825084296122464]
# 收益最大值为: 0.5956249931745368

ind = [0.6064531271027143, 1.1469476180530012, 1.2453036834458149, -1.657479142147305, -1.2606158379235208, 0.3857420334185466, -0.5111631710731213, 1.4795654027014211, -0.9825084296122464]

# #训练集表现
my_data=factor(my_data)
my_data_2 =bound_1(my_data,ind)
data_all = pd.merge(my_data_1,my_data_2,on='date')
my_signal = signal(data_all)
my_position = position(my_signal)
res_my_data, performance_df ,hold_cumu_r= statistic_performance(my_position, 
                                                   data_period=1, 
                                                   is_consider_open=True,
                                                   is_consider_stop = True,                       
                                                   point=0.4,
                                                   is_daily_close=True,
                                                   comm=0.00003)
print(performance_df)
#全集表现
my_data_1d = factor(my_data_1d)
my_data_2 =bound_1(my_data_1d,ind)
data_all_time = pd.merge(my_data_2m,my_data_2,on='date')
my_signal = signal(data_all_time)
my_position = position(my_signal)
res_my_data, performance_df ,hold_cumu_r= statistic_performance(my_position, 
                                                   data_period=1, 
                                                   is_consider_open=True,
                                                   is_consider_stop = True,                       
                                                   point=0.4,
                                                   is_daily_close=True,
                                                   comm=0.00003)
print(performance_df)

                0
累计收益       59.56%
多仓次数           34
多仓成功次数         25
多仓胜率       73.53%
多仓平均持有期     59.18
空仓次数           42
空仓成功次数         24
空仓胜率       57.14%
空仓平均持有期     55.05
周期胜率       52.32%
最大回撤        3.37%
年化收益/最大回撤    9.60
年化收益       32.32%
年化标准差       7.88%
年化夏普         4.10
                0
累计收益       78.30%
多仓次数           43
多仓成功次数         31
多仓胜率       72.09%
多仓平均持有期     57.42
空仓次数           58
空仓成功次数         32
空仓胜率       55.17%
空仓平均持有期     49.14
周期胜率       52.53%
最大回撤        3.37%
年化收益/最大回撤    6.92
年化收益       23.32%
年化标准差       6.56%
年化夏普         3.55


In [177]:
res_my_data

,date,time,dates,open,close,high,low,B,low_B,high_B,...,signal,stop_or_not,position,hold_in_price,hold_out_price,hold_r,hold_win,hold_cumu_r,drawdown,ex_hold_r
0,2010-04-30,09:17:00,2010/4/30 9:17,3121.069,3120.926,3123.026,3114.525,10.424508,3110.644492,3131.493508,...,None,False,0.0,0.0,0.0,0.0,False,0.00000,0.000000,-8.333333e-08
1,2010-04-30,09:19:00,2010/4/30 9:19,3120.851,3122.523,3124.567,3120.167,10.424508,3110.644492,3131.493508,...,None,False,0.0,0.0,0.0,0.0,False,0.00000,0.000000,-8.333333e-08
2,2010-04-30,09:21:00,2010/4/30 9:21,3122.811,3121.291,3123.535,3120.112,10.424508,3110.644492,3131.493508,...,None,False,0.0,0.0,0.0,0.0,False,0.00000,0.000000,-8.333333e-08
3,2010-04-30,09:23:00,2010/4/30 9:23,3120.751,3118.206,3121.857,3116.785,10.424508,3110.644492,3131.493508,...,None,False,0.0,0.0,0.0,0.0,False,0.00000,0.000000,-8.333333e-08
4,2010-04-30,09:25:00,2010/4/30 9:25,3117.972,3120.714,3120.830,3117.453,10.424508,3110.644492,3131.493508,...,None,False,0.0,0.0,0.0,0.0,False,0.00000,0.000000,-8.333333e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85883,2013-01-31,14:53:00,2013/1/31 14:53,2701.257,2703.917,2705.317,2701.007,10000.000000,-7291.752000,12708.248000,...,None,False,0.0,0.0,0.0,0.0,False,0.78303,0.005407,-8.333333e-08
85884,2013-01-31,14:55:00,2013/1/31 14:55,2704.064,2704.509,2704.901,2703.163,10000.000000,-7291.752000,12708.248000,...,None,False,0.0,0.0,0.0,0.0,False,0.78303,0.005407,-8.333333e-08
85885,2013-01-31,14:57:00,2013/1/31 14:57,2704.365,2704.055,2705.622,2703.911,10000.000000,-7291.752000,12708.248000,...,None,False,0.0,0.0,0.0,0.0,False,0.78303,0.005407,-8.333333e-08
85886,2013-01-31,14:59:00,2013/1/31 14:59,2704.058,2704.102,2704.782,2703.682,10000.000000,-7291.752000,12708.248000,...,None,False,0.0,0.0,0.0,0.0,False,0.78303,0.005407,-8.333333e-08
